<a href="https://colab.research.google.com/github/keivanipchihagh/Google-ML-Crash-Course/blob/master/Validation%20and%20Test%20Sets/My_code_Validation_and_Test_Sets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Import modules
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
# Load datasets
train_data = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")
test_data = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv")
train_data.head()

In [3]:
# Scale
scale_factor = 1000.0

train_data['median_house_value'] /= scale_factor
test_data['median_house_value'] /= scale_factor

# Shuffle
train_data = train_data.reindex(np.random.permutation(train_data.index))

In [4]:
# Build the model
def build_model(learning_rate):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(units = 1, input_shape = (1,)))
  
  model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate), 
                loss = tf.keras.losses.mean_absolute_error, 
                metrics = [tf.keras.metrics.RootMeanSquaredError()])
  
  return model

In [5]:
# Train the model
def train_model(model, dataframe, feature, label, epochs, verbose, batch_size = None, validation_split = 0.1):
  history = model.fit(x = dataframe[feature], 
                      y = dataframe[label], 
                      batch_size = batch_size, 
                      epochs = epochs, 
                      validation_split = validation_split, 
                      verbose = verbose)

  return history.epoch, pd.DataFrame(history.history)["root_mean_squared_error"], history.history   

In [6]:
# Plot the statistics
def plot(epochs, training, validation):
  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs[1:], training[1:], label="Training Loss")
  plt.plot(epochs[1:], validation[1:], label="Validation Loss")
  plt.legend()
  plt.show()

In [ ]:
# Summon up
learning_rate = 0.08
epochs = 30
batch_size = 100
validation_split = 0.1
feature="median_income"
label="median_house_value"
verbose = False

model = build_model(learning_rate)
epoch, rmse, history = train_model(model, train_data, feature, label, epochs, verbose, batch_size, validation_split)
print(history.keys())
plot(epoch, history['root_mean_squared_error'], history['val_root_mean_squared_error'])

In [ ]:
# Evaluate
results = model.evaluate(x = test_data[feature], y = test_data[label], batch_size = batch_size)
print(results)